In [ ]:
import os
import pickle
import pandas as pd
df = pickle.load(open('SFP_Train_Quadrant_df.pkl','rb'))



In [ ]:
submission_script = """#!/bin/bash
#SBATCH -A {your account name}
#SBATCH -C gpu
#SBATCH -q regular
#SBATCH -J x{expert_id}
#SBATCH -t {time}
#SBATCH -N 1
#SBATCH --ntasks-per-node=1
#SBATCH -c 32
#SBATCH --gpus-per-task=4

export SLURM_CPU_BIND="cores"

module load conda
conda activate myenv
srun python SuperFP_Expert_Index_Finetuning.py {expert_id}

"""

In [ ]:

final_script = submission_script.format(
time = '12:00:00',
expert_id = 0,
)
print(final_script)


In [ ]:
#Parallel training upon submission

submission_command = """sbatch {filename}"""
# you need to create a "Job_Logs" directory
os.chdir('Job_Logs')

for expert_id, counts in dict(df['quadrant'].value_counts()).items():
    #print(expert_id, counts)
    if counts < 400:
        use_time = '1:00:00'
    elif counts < 800:
        use_time = '2:00:00'
    elif counts < 1600:
        use_time = '4:00:00'
    elif counts < 3200:
        use_time = '8:00:00'
    else:
        print('Beyond 3200 entries is going to take more than 8 hrs to finish 10 epochs of training, you need to configure this')
        break
    
    
    final_script = submission_script.format(
    time = use_time,
    expert_id = expert_id,
    )
    # You need a create a "Submission_Scripts" directory
    sub_file_name = 'Submission_Scripts/'+ str(expert_id) + '.sub'
    with open(sub_file_name, 'w') as f:
        f.write(final_script)
    os.system(submission_command.format(filename = sub_file_name))
    
    